# GSoC Advances: Huckel Parameters
## Andrés Gómez
###### Issue: Huckel Parameters 

In this document is explained the additions implemented to the moha program. 
The problems addressed are the first 2 on the Issue Huckel Parameters on the GitHub repository: 

"""

We need to be able to

- [x] parse atom types
- [x] assign appropriate α and β values based on Rauk's atom types (attached, page 94).
- [ ] compute appropriate α and β values using distance/overlap.

"""

In this manner the principal commits on this fork are the following:

- atom_types list construction (correction)
- Added coordination functionality to get_atom_type function in utils.py and compatibility with API.
- Added dictionaries including alpha_x and  kxy values from Rauk's table
- Implemented atom_dictionary and bond_dictionary construction from the atom_types list from Rauk's table.
- Implemented Huckel parameters function in the API and use in the HamPPP class. 
- Added the functionality to allow the user to insert atom_dictionary and bond_dictionary. 




First to check that the program works for the usual cases, here I present the run for the same example given in the "Demonstration.ipynb" for Polyene

In [1]:
import numpy as np

### Function for printing the hamiltonian
def print_hamiltonian(h0, h1, h2):
    np.set_printoptions(precision=3)
    print('h0=\n',"%0.3f" % h0,'\n')        
    print('h1=\n',h1,'\n')
    print('h2=\n',h2,'\n')

### Gamma value generator for polyene
def generate_gamma(norb, b, gamma0): 
    ang_to_bohr = 1.889726
    har_to_ev = 27.211396
    b_ieV = b*ang_to_bohr/har_to_ev

    gamma = np.zeros((norb,norb))
    for u in range(norb):
        for v in range(norb):
            duv = b*(np.sin(abs(u-v)*np.pi/norb)/np.sin(np.pi/norb))
            gamma[(u,v)] = 1/(1/gamma0 + duv)

    return gamma 
    
### Building integral arrays for benzene
norb = 6
b = 1.4
gamma0 = 10.84
beta = -2.5

gamma = generate_gamma(norb, b, gamma0)
gamma0 = gamma[0,0]

import sys  
sys.path.insert(0, '../')

from moha import HamPPP

polyene = HamPPP([(f"C{i}", f"C{i + 1}", 1) for i in range(1, norb)] + [(f"C{norb}", f"C{1}", 1)],
                      alpha=0, beta=beta, gamma=gamma, charges=np.ones(6),
                      u_onsite=np.array([0.5*gamma0 for i in range(norb + 1)]))

h0 = polyene.generate_zero_body_integral()
h1 = polyene.generate_one_body_integral(dense=True, basis='spatial basis')
h2 = polyene.generate_two_body_integral(dense=False, basis='spatial basis')

print_hamiltonian(h0, h1, h2)

ValueError: not enough values to unpack (expected 4, got 3)

All looking good so far. 



The new functionalities are that now the one body term generation (Huckel Model) can take into account different alpha and beta values depending on the atoms in the system. In this manner, from the connectivity system definition the atom_types list is constructed with the atomic symbols present in the system. If the atom_dictionary or bond_dictionary parameter is given, this are taken for the Huckel model construction, if not, this dictionaries are constructed from the atom_types list usin Rauks's parameters.

List of Rauk's parameters: 

<img src="img/Raukstable.png" class="center" style="width: 1000px;"/>


And stored (extracted from assign_Huckel_parameters method):








In [2]:
hx_dictionary = { 
            "C":  0.0, 
            "B":  0.45, 
            "N2":-0.51,
            "N3":-1.37,
            "O1":-0.97,
            "O2":-2.09,
            "F": -2.71,
            "Si":  0.0,
            "P2": -0.19,
            "P3": -0.75,
            "S1": -0.46,
            "S2": -1.11,
            "Cl": -1.48
            }
        
        #kxy elements
kxy_matrix_1 = np.array([
        [-1.0  , 0.    ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   , 0.    ,  0.    , 0.   ],
        [-0.73 , -0.87 ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   , 0.    ,  0.    , 0.   ],
        [-1.02 , -0.66 , -1.09 ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   , 0.    ,  0.    , 0.   ],
        [-0.89 , -0.53 , -0.99 , -0.98 ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   , 0.    ,  0.    , 0.   ],
        [-1.06 , -0.60 , -1.14 , -1.13 , -1.26 ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   , 0.    ,  0.    , 0.   ],
        [-0.66 , -0.35 , -0.80 , -0.89 , -1.02 , -0.95 ,  0.   ,  0.   ,  0.   ,  0.   , 0.    ,  0.    , 0.   ],
        [-0.52 , -0.26 , -0.65 , -0.77 , -0.92 , -0.94 , -1.04 ,  0.   ,  0.   ,  0.   , 0.    ,  0.    , 0.   ],
        [-0.75 , -0.57 , -0.72 , -0.43 , -0.65 , -0.24 , -0.17 , -0.64 ,  0.   ,  0.   , 0.    ,  0.    , 0.   ],
        [-0.77 , -0.53 , -0.78 , -0.55 , -0.75 , -0.31 , -0.21 , -0.62 , -0.63 ,  0.   , 0.    ,  0.    , 0.   ],
        [-0.76 , -0.54 , -0.81 , -0.64 , -0.82 , -0.39 , -0.22 , -0.52 , -0.58 , -0.63 , 0.    ,  0.    , 0.   ],
        [-0.81 , -0.51 , -0.83 , -0.68 , -0.84 , -0.43 , -0.28 , -0.61 , -0.65 , -0.65 , -0.68 ,  0.    , 0.   ],
        [-0.69 , -0.44 , -0.78 , -0.73 , -0.85 , -0.54 , -0.32 , -0.40 , -0.48 , -0.60 , -0.58 , -0.63  , 0.   ],
        [-0.62 , -0.41 , -0.77 , -0.80 , -0.88 , -0.70 , -0.51 , -0.34 , -0.35 , -0.55 , -0.52 , -0.59  ,-0.68 ],
        ])
kxy_matrix = np.minimum(kxy_matrix_1, kxy_matrix_1.T) #Symmetric

Additionally, as there are cases in which the coordination varies for a given atom, the connectivity input in this case can be given by adding a _coordination term:

In [6]:
#[('atomsymbolsite_coordinationnumber','atomsymbolsite_coordinationnumber',bond),...]

#Same old format works
connectivity= [('C1', 'C2', 1), ('C2', 'C3', 1), ('C3', 'C1', 1)]

#Including coordination (Must be in Rauks table)
connectivity= [('N1_2', 'O2_1', 1), ('O2_1', 'P3_2', 1), ('P3_2', 'N1_2', 1)]

In [19]:
import sys 
import numpy as np
sys.path.insert(0, '../')
def print_hamiltonian(h1):
    np.set_printoptions(precision=3)      
    print('h1=\n',h1,'\n')
from moha import HamHuck



Then, presenting some examples (imaginary molecules): 

- C_Cl_F_Si_C molecule (No coordination, parameters not given)


In [27]:
C_Cl_F_Si = HamHuck([('C1', 'Cl2', 1), ('Cl2', 'F3', 1),
                    ('F3', 'Si4', 1), ('Si4', 'C1', 1)])
h1 = C_Cl_F_Si.generate_one_body_integral(dense=True, basis='spatial basis')
print_hamiltonian(h1)


h1=
 [[-0.414 -0.033  0.    -0.04 ]
 [-0.033 -0.493 -0.027  0.   ]
 [ 0.    -0.027 -0.558 -0.009]
 [-0.04   0.    -0.009 -0.414]] 



- N2_O1_P2 molecule (coordination and parameters not given )

In [29]:
N2_O1_P2 = HamHuck([('N1_2', 'O2_1', 1), ('O2_1', 'P3_2', 1), ('P3_2', 'N1_2', 1)])
h1 = N2_O1_P2.generate_one_body_integral(dense=True, basis='spatial basis')
print_hamiltonian(h1)


h1=
 [[-0.441 -0.061 -0.042]
 [-0.061 -0.466 -0.04 ]
 [-0.042 -0.04  -0.424]] 



Now, if the user wants to specify the parameters the must be given as dictionaries in the following manner:

Taking for example the C_Cl_F_Si case:



In [30]:
C_Cl_F_Si = HamHuck([('C1', 'Cl2', 1), ('Cl2', 'F3', 1),
                    ('F3', 'Si4', 1), ('Si4', 'C1', 1)])
h1 = C_Cl_F_Si.generate_one_body_integral(dense=True, basis='spatial basis')
print('From Rauks')
print_hamiltonian(h1)
print('')

atomdic = {'C':-10,'Cl':-20,'F':-30,'Si':-40}
betadic = {'CCl':-1,'ClF':-2,'FSi':-3,'SiC':-4}
C_Cl_F_Si = HamHuck([('C1', 'Cl2', 1), ('Cl2', 'F3', 1),
                    ('F3', 'Si4', 1), ('Si4', 'C1', 1)], 
                    atom_dictionary=atomdic, bond_dictionary=betadic)
h1 = C_Cl_F_Si.generate_one_body_integral(dense=True, basis='spatial basis')
print('Given parameters')
print_hamiltonian(h1)


From Rauks
h1=
 [[-0.414 -0.033  0.    -0.04 ]
 [-0.033 -0.493 -0.027  0.   ]
 [ 0.    -0.027 -0.558 -0.009]
 [-0.04   0.    -0.009 -0.414]] 


Given parameters
h1=
 [[-10.  -1.   0.  -4.]
 [ -1. -20.  -2.   0.]
 [  0.  -2. -30.  -3.]
 [ -4.   0.  -3. -40.]] 



Wolfsberrg-Helmholz approximation

The program results for the overlap was verified by reproducing the tables presented in the paper:

<img src="img/TableXX.png" class="center" style="width: 1000px;"/>

<img src="img/TableGen.png" class="center" style="width: 1000px;"/>


In [2]:
from moha import HamHuck
import sys
import numpy as np
sys.path.insert(0, '../')


def print_hamiltonian(h1):
    np.set_printoptions(precision=3)
    print('h1=\n', h1, '\n')


C_Cl_F_Si = HamHuck([('C1', 'Cl2', 1, 1), ('Cl2', 'F3', 1, 3),
                    ('F3', 'Si4', 1, 2), ('Si4', 'C1', 1, 3)])
h1 = C_Cl_F_Si.generate_one_body_integral(dense=True, basis='spatial basis')
print_hamiltonian(h1)


h1=
 [[-0.414 -0.754  0.    -0.525]
 [-0.754 -0.477 -0.717  0.   ]
 [ 0.    -0.717 -0.64  -0.537]
 [-0.525  0.    -0.537 -0.3  ]] 

